# Web Scraping Instagram with Selenium - UPDATED!

### AUTOMATING IMAGE EXTRACTION

This Notebook is an <b>upgraded</b> version of the original Web Scraping Instagram with Selenium notebook.
<br>
This code is asjusted to fit a wider variety of users and has solutions for the issues that arrised in the comment section on YouTube/ issues section on Github.
<br>
Also, this code is extracting the <b>full size images</b> and not the <b>thumbnails</b>, and it's a <b>100% automated</b>!

Please let me know if you have any other problems that you haven't found a solution for in the comment section of the Youtube tutorial:
<br>
https://youtu.be/iJGvYBH9mcY

In [4]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time

import os
import wget
import errno
import csv


In [5]:
user1 = ""
password1 = ""

## Download ChromeDriver
Now we need to download latest stable release of ChromeDriver from:
<br>
https://chromedriver.chromium.org/
<br>
## Log In to Your Instagram Account

## Handle Alerts

you might only get a single alert, or you might get 2 of them
<br>
please adjust the cell below accordingly

In [16]:
driver = webdriver.Chrome("E:\chrome\chromedriver.exe")

def logIn(user1, password1):
    
    print('start log in to the account ')
    #specify the path to chromedriver.exe (download and save on your computer)
    #open the webpage
    driver.get("http://www.instagram.com")

   
    username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
    password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

    #enter username and password
    username.clear()
    password.clear()
    
    username.send_keys(user1)
    password.send_keys(password1)

    #target the login button and click it
    button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, "button[type='submit']"))).click()

    #We are logged in!

  
    time.sleep(5)
    
    alert = WebDriverWait(driver, 15).until(EC.element_to_be_clickable(
        (By.XPATH, '//button[contains(text(), "Not Now")]'))).click()
    #alert2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable(
    #(By.XPATH, '//button[contains(text(), "Not Now")]'))).click()

<ipython-input-16-d9e1e2e78cfb>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("E:\chrome\chromedriver.exe")


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 93
Current browser version is 96.0.4664.45 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00AFD403+2479107]
	Ordinal0 [0x00A97D51+2063697]
	Ordinal0 [0x009A20C8+1056968]
	Ordinal0 [0x009C043A+1180730]
	Ordinal0 [0x009BC355+1164117]
	Ordinal0 [0x009B9B7F+1153919]
	Ordinal0 [0x009E9CBF+1350847]
	Ordinal0 [0x009E992A+1349930]
	Ordinal0 [0x009E583B+1333307]
	Ordinal0 [0x009C1F94+1187732]
	Ordinal0 [0x009C2DE9+1191401]
	GetHandleVerifier [0x00C7F3F6+1531734]
	GetHandleVerifier [0x00D2E9AE+2249998]
	GetHandleVerifier [0x00B839AB+501003]
	GetHandleVerifier [0x00B82A29+497033]
	Ordinal0 [0x00A9D11D+2085149]
	Ordinal0 [0x00AA11C8+2101704]
	Ordinal0 [0x00AA1302+2102018]
	Ordinal0 [0x00AAAA71+2140785]
	BaseThreadInitThunk [0x7596FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77437A9E+286]
	RtlGetAppContainerNamedObjectPath 

## Search for a certain hashtag

In [7]:
def findImage(name):
    print('finding the images ')
    #target the search input field
    searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,\
                                    "//input[@placeholder='Search']")))
    searchbox.clear()

    #search for the hashtag cat
    keyword = name
    searchbox.send_keys(keyword)
    
    #FIXING THE DOUBLE ENTER
    time.sleep(5) # Wait for 5 seconds
    
    searchbox.send_keys(Keys.ENTER)
    time.sleep(1)
    searchbox.send_keys(Keys.ENTER)
    """my_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
        (By.XPATH, "//a[contains(@href= '/explore/tags/" +keyword[1:]+"/')]"))).click()"""

    
    

## Scroll Down
<br>
Increase n_scrolls to select more photos (depending on screen resolution)
<br>
<b>Example:</b>
<br>
<ul>
    <li>2 scrolls cover approx. 35 photos</li>
    <li>3 scrolls cover approx. 45 photos</li>
</ul>

In [8]:
def scroll (scrolls):
    print('satrt scrolling down ')
    #scroll down 2 times
    #increase the range to sroll more
    n_scrolls = scrolls
    for j in range(0, n_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

In [9]:
def targetlink():
    print('opening the link ')
    #target all the link elements on the page
    anchors = driver.find_elements_by_tag_name('a')
    anchors = [a.get_attribute('href') for a in anchors]
    #narrow down all links to image links only
    anchors = [a for a in anchors if str(a).startswith("https://www.instagram.com/p/")]

    print('Found ' + str(len(anchors)) + ' links to images')
    anchors[:5]
    a = anchors 
    print(len(a))
    return a

In [10]:
def extractImages(anchors):
    print('extracting the images ')
    images = []

    #follow each image link and extract only image at index=1
    for a in anchors:
        driver.get(a)
        time.sleep(5)
        img = driver.find_elements_by_tag_name('img')
        img = [i.get_attribute('src') for i in img]
        images.extend(img)
        a = images
    print('all images are: ', len(images))
    return a  
    

## Save images to computer

First we'll create a new folder for our images somewhere on our computer.
<br>
Then, we'll save all the images there.

In [11]:

def createfolder(folderName): 
    print('creating the folder if it is not found ')
    path = os.getcwd()
    path = os.path.join(path, folderName[1:] + "s")

    #create the directory
    try:
        os.mkdir(path)
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise
        pass

    return path

In [12]:
#download images
def imagedawnloader(path,images): 
    print('start the download ')
    counter = 0
    for image in images:
        save_as = os.path.join(path,  str(counter) + '.jpg')
        wget.download(image, save_as)
        counter += 1

In [13]:
logIn(user1, password1)
tagesname= ['#cheetah',  '#lion','#griaffe','#Snake']
with open ('data.csv', 'w') as f: 
        writer = csv.writer(f)
        writer.writerow(["tag","link"])
for tag in tagesname: 
    print(tag)
    findImage(tag)
    scroll (20)
    anchors = targetlink()
    images= extractImages(anchors)
    path =createfolder(tag)
    imagedawnloader(path,images)
    with open ('data.csv', 'a') as f: 
        writer = csv.writer(f)
        for img in range(len(images))  : 
            
            writer.writerow([tag[1:],images[img]])


NameError: name 'logIn' is not defined

,'#Crocodile', '#Bear.wild', '#Snake.wild', '#Whale.wild', '#Tiger.wild', '#Lions.wild', '#Eagle.wild','#vulture.wild','#kangaroo.wild', '#monkey.wild' ,' #zebra.wild', '#hippo.wild', '#bull.wild', '#elephant.wild', '#gorilla.wild','#griaffe.wild', '#Ostrich.wild', '#antelope.wild'

 with open ('data.csv', 'a') as f: 
        writer = csv.writer(f)
        for img in range(len(images))  : 
            
            writer.writerow([tag[1:],images[img]])
